In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, Layout
from IPython.display import display
from dash import Dash, dcc, html, Input, Output
import os
from threading import Timer
import webbrowser
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import re
from functools import reduce

In [92]:
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)

In [93]:
def open_browser():
    if not os.environ.get("WERKZEUG_RUN_MAIN"):
        webbrowser.open_new('http://127.0.0.1:8080/')

In [94]:
linkedin_job_postings = pd.read_csv("..\Data\linkedin_jobs_and_skills\linkedin_job_postings_cleaned.csv")

df_twc = pd.read_csv('..\Data\Top_Worlds_Companies\companies.csv')

df_ldd = pd.read_excel('..\Data\LinkedIn_DigitalData\public_use-industry-skills-needs.xlsx', sheet_name=['Industry Skills Needs'])['Industry Skills Needs']
df = pd.read_csv('..\Data\linkedin_jobs_and_skills\job_skills.csv')
linkedin_job = pd.merge(linkedin_job_postings, df, on = 'job_link', how = 'inner').dropna()

df = pd.read_csv('..\Data\Jobs_Data_Worldwide\globe_all_countries_jobs_data.csv', encoding='latin1')
df.columns = df.columns.str.strip()

df_growth = pd.read_excel('..\Data\LinkedIn_DigitalData\public_use-industry-employment-growth.xlsx', sheet_name=['Growth from Industry Transition'])['Growth from Industry Transition']
df_growth = df_growth.rename(columns={'growth_rate_2015': '2015', 'growth_rate_2016': '2016', 'growth_rate_2017': '2017', 'growth_rate_2018': '2018', 'growth_rate_2019': '2019'})

df_mig = pd.read_excel('..\Data\LinkedIn_DigitalData\public_use-talent-migration.xlsx', sheet_name=['Country Migration', 'Industry Migration', 'Skill Migration'])

## Part 1

In [95]:
# Function to identify top hiring companies for a specific job position
def top_hiring_companies(jobs_df, country, top_n=20):
    
    # Filter jobs by country
    jobs_df = jobs_df[jobs_df['search_country'] == country]

    # Count occurrences of each company
    company_counts = jobs_df['company'].value_counts().head(top_n)
    
    return company_counts

# Create initial plot
default_job_position = ''
default_country = 'United States'
initial_jobs_df = linkedin_job_postings[linkedin_job_postings['job_title'].str.contains(default_job_position.lower(), case=False)]
top_companies = top_hiring_companies(initial_jobs_df, default_country)

fig1 = go.FigureWidget(
    data=[
        go.Bar(
            y=top_companies.index, 
            x=top_companies.values,
            text=top_companies.values,
            orientation='h',
        )
    ]
).update_layout(
    title=f"Top Companies hiring in \'{default_country}\'",
    yaxis_title="Company",
    xaxis_title="Number of Job Postings",
    yaxis=dict(autorange="reversed"),
    height=1000,
    title_x=0.5,
)

# Function to update plot based on filtered job position
def update_fig1_plot(search_value, country_value):
    if search_value:
        filtered_jobs_df = linkedin_job_postings[linkedin_job_postings['job_title'].str.contains(search_value.lower(), case=False)]
        if len(filtered_jobs_df) > 0:
            top_companies = top_hiring_companies(filtered_jobs_df, country_value)
            fig1.data[0].y = [(i[:30] + '..') if len(i) > 30 else i for i in top_companies.index]
            fig1.data[0].x = top_companies.values
            fig1.data[0].text=top_companies.values.astype(float)
            fig1.data[0].orientation='h'
            fig1.update_layout(
                title=f"Top companies hiring for \'{search_value}\' in \'{country_value}\'",
                yaxis=dict(autorange="reversed"),
                title_x=0.5
            )
        else:
            fig1.data[0].y = []
            fig1.data[0].x = []
            fig1.update_layout(title=f"No matching job positions for {search_value}",title_x=0.5,title_font_color="red")
    else:
        top_companies = top_hiring_companies(linkedin_job_postings, country_value)
        fig1.data[0].y = [(i[:30] + '..') if len(i) > 30 else i for i in top_companies.index]
        fig1.data[0].x = top_companies.values
        fig1.data[0].text=top_companies.values.astype(float)
        fig1.data[0].orientation='h'
        fig1.update_layout(
            title=f"Top companies hiring in \'{country_value}\'",
            yaxis=dict(autorange="reversed"),
            title_x=0.5
        )

## Part 2

In [96]:
country_list = linkedin_job_postings['search_country'].unique().tolist()
country_list.sort()

linkedin_job_postings_by_country_dict = {}

for country in country_list:
    linkedin_job_postings_by_country_dict[country] = linkedin_job_postings[linkedin_job_postings['search_country']==country]

top = 10
top_cities_dict = {}

for country in country_list:
    top_cities_dict[country] = linkedin_job_postings_by_country_dict[country]['search_city'].value_counts().head(top)

fig_dict = {}

chart_colors = np.array(['rgb(239,85,59)', 'rgb(99,110,250)', 'rgb(254,203,82)', 'rgb(255,151,255)', 'rgb(182,232,128)',
                'rgb(255,102,146)', 'rgb(25,211,243)', 'rgb(255,161,90)', 'rgb(171,99,250)', 'rgb(0,204,150)'])

chart_colors_dict = {}

for country in country_list:
    chart_colors_dict[country] = chart_colors.copy()
    np.random.shuffle(chart_colors)

for country in country_list:
    total = top_cities_dict[country].values.sum()
    fig_dict[country] = go.Figure(
                            data=[
                                go.Pie(
                                    labels=top_cities_dict[country].index, 
                                    values=top_cities_dict[country].values,
                                    textinfo='label+value',
                                    insidetextorientation='horizontal',
                                    showlegend=False,
                                    pull=top_cities_dict[country].values / total,
                                    marker=dict(colors=chart_colors_dict[country]),
                                )
                            ]
                        )
    fig_dict[country].update_layout(
        title_text=f"Top {top} cities with highest jobs in {country}",
        title_x=0.5,
        title_y=0.95,
    )

fig2 = make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=[f"Top {top} cities with highest jobs in {country}" for country in country_list],
    specs=[[{"type": "sunburst"}, {"type": "sunburst"}], [{"type": "sunburst"}, {"type": "sunburst"}]],
    horizontal_spacing=0.2,
    vertical_spacing=0.1,
)

for country in country_list:
    fig2.add_trace(fig_dict[country].data[0], row=country_list.index(country)//2+1, col=country_list.index(country)%2+1)

fig2.update_layout(
    height=1200, 
    # width=1000, 
    # title_text="Top 10 cities with highest jobs in each country",
    # title_x=0.5,
    # title=dict(
    #     font=dict(
    #         size=24,  # Adjust text size as needed
    #         color="black",  # Adjust color as needed
    #     )
    # ),
    #     margin=dict(l=50, r=50, t=100, b=50),  # Adjust margins as needed
        # spacing=dict(row=0.2, column=0.2)
)

## Part 3

In [97]:
# Top world's companies
def contains_numeric(string):
    pattern = r'\d'
    match = re.search(pattern, string)
    if match:
        return np.nan
    elif string == 'Ahmedabad':
        return np.nan
    else:
        return string

def string_to_num(amount):
    r = 0
    try:
        if amount[-1]=='k':
            r = 1000*float(amount[:-1])
        else:
            r = float(amount)
        return r
    except ValueError:
        return np.nan
    
df_twc['desc'] = df_twc['Description'].str.split('|').str[0]
df_twc['sector'] = df_twc['desc'].apply(contains_numeric)

df_twc.drop('desc',axis=1, inplace=True)

df_twc['HRF'] = df_twc['Highly_rated_for'].str.split(', ')
df_twc['Avg_salary'] = df_twc['Avg_salary'].apply(string_to_num)
df_twc['Total_jobs_available'] = df_twc['Total_jobs_available'].apply(string_to_num)
df_twc['Interviews_taken'] = df_twc['Interviews_taken'].apply(string_to_num)
hrf_ser = df_twc['HRF'].dropna()

union_HRF = reduce(set.union, map(set, hrf_ser))

In [98]:
def update_Sect_Ratings(sector, crit):
    # Apply filters to the data
    filtered_df = df_twc.copy()
    filtered_df.dropna(subset=['sector','HRF'], inplace=True)

    if sector:
        filtered_df = filtered_df[filtered_df["sector"].isin(sector)]
    if crit:
        filtered_df = filtered_df[filtered_df["HRF"].apply(lambda x: bool(set(x) & set(crit)))]

    # Create the bar chart
    full_fig = make_subplots(rows=2, cols=1,)
    fig1 = px.bar(
        filtered_df.sort_values("Avg_salary", ascending=False).head(20),
        x="Company_name",
        y="Avg_salary",
        color="Ratings",
        hover_data=["Avg_salary"],
    )
    fig1.update_xaxes(showgrid=False)
    fig1.update_yaxes(showgrid=False)
    full_fig.update_yaxes(title_text="Average salary", row=1, col=1)
    full_fig.update_layout(
        coloraxis_colorbar=dict(len=0.4, x=1.1, y=0.7, title='Ratings'),
        coloraxis_colorscale='ice',
    )
    fig2 = px.scatter(
        filtered_df.sort_values("Avg_salary", ascending=False).head(20),
        x='Total_jobs_available', y='Interviews_taken', 
        color='Ratings',size = 'Ratings',hover_data=["Company_name"]
    )
    full_fig.update_xaxes(title_text="Number of jobs", row=2, col=1)
    full_fig.update_yaxes(title_text="Number of interviews", row=2, col=1)
    full_fig.add_trace(fig1.data[0], row=1,col=1)
    full_fig.add_trace(fig2.data[0], row=2, col=1)
    full_fig.update_layout(
        # width=1200,
        height=700,
        margin=dict(l=20, r=20, t=40, b=20),
        font=dict(color=colors['text']),
        plot_bgcolor=colors['background'], paper_bgcolor=colors['background']
    )
    return full_fig

## Part 4

In [99]:
def update_style(section, skill, selected_node):
    filtered_df = df_ldd[df_ldd['isic_section_name']==section]
    if skill:
        filtered_df = filtered_df[filtered_df["skill_group_name"].isin(skill)]
    industries = [
        {
            "data":{"id": iname+"i", "label": iname, "type":'industry'}
        }
        for iname in filtered_df['industry_name'].unique()
    ]
    skills = [
        {
            "data":{"id": sname+"s", "label": sname, "type":'skill'}
        }
        for sname in filtered_df['skill_group_name'].unique()
    ]
    nodes = industries+skills
    edges = [
        {"data": {"source": source+"i", "target":target+"s"}}
        for source, target in list(zip(filtered_df['industry_name'], filtered_df['skill_group_name']))
    ]
    elements = nodes + edges
    if selected_node is None:
        return elements, []

    selected_node_id = selected_node['id']
    neighborhood = []
    for edge in edges:
        edge_data = edge.get('data', {})
        source = edge_data.get('source')
        target = edge_data.get('target')
        if source == selected_node_id:
            neighborhood.append(target)
        elif target == selected_node_id:
            neighborhood.append(source)

    # Define the stylesheet to highlight the selected node and its immediate neighbors
    stylesheet = [
        {
            'selector': 'node',
            'style': {
                'background-color': 'grey',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'node[type = \'industry\']',
            'style': {
                'background-color': '#ff66cc',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'edge',
            'style': {
                'line-color': 'grey',  # Default color for edges
                'opacity':'0.15'
            }
        },
        {
            'selector': 'node[id = "{}"]'.format(selected_node_id),
            'style': {
                'background-color': 'blue',  # Color for selected node
                'opacity': '1.0'
            }
        },
        {
            'selector': 'edge[source = "{}"]'.format(selected_node_id),
            'style': {
                'line-color': 'blue',  # Color for edges connected to selected node
                'opacity': '1.0'
            }
        },
        {
            'selector': 'edge[target = "{}"]'.format(selected_node_id),
            'style': {
                'line-color': 'blue',  # Color for edges connected to selected node
                'opacity': '1.0'
            }
        }
    ]
    for nd in neighborhood:
        stylesheet.append({
            'selector': 'node[id = "{}"]'.format(nd),
            'style': {
                'background-color': 'red',  # Color for edges connected to selected node
                'opacity': '1.0'
            }})
    return elements,stylesheet

## Part 5

In [100]:
country_options = [{'label': country, 'value': country} for country in linkedin_job_postings['search_country'].unique()]

default_country = 'United States'
default_city = 'New York'

filtered_df = linkedin_job_postings[linkedin_job_postings['search_country'] == default_country][linkedin_job_postings['search_city'] == default_city][['search_position']]
data_df = filtered_df.groupby('search_position').size().reset_index(name='count').sort_values('count', ascending=False).reset_index(drop=True)

fig5 = px.histogram(
            data_df.head(15),
            x='search_position',
            y='count',
            text_auto=True,
        ).update_layout(
            title=f'Top 20 job positions users search for in {default_city}, {default_country}',
            title_x=0.5,
            xaxis_title='Job Position',
            yaxis_title='Total Job Postings',
            xaxis={'categoryorder':'total descending'}
        )

def update_histogram(search_country, search_city):
    filtered_df = linkedin_job_postings[linkedin_job_postings['search_country'] == search_country][linkedin_job_postings['search_city'] == search_city][['search_position']]
    data_df = filtered_df.groupby('search_position').size().reset_index(name='count').sort_values('count', ascending=False).reset_index(drop=True)

    fig5.data[0].x = data_df.head(15)['search_position']
    fig5.data[0].y = data_df.head(15)['count']

    fig5.update_layout(
            title=f'Top 20 job positions users search for in {search_city}, {search_country}',
            xaxis_title='Job Position',
            yaxis_title='Total Job Postings',
            xaxis={'categoryorder':'total descending'}
        )
    return fig5

C:\Users\ashwa\AppData\Local\Temp\ipykernel_11844\3207145753.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



## Part 6

In [101]:
selected_company = linkedin_job['company'].head(100).tolist()

def update_chart(company):
    company_data = linkedin_job[linkedin_job['company'] == company]
    job_counts = company_data['search_position'].value_counts().head(15)

    # Create a bar graph
    bar_data = go.Bar(
        x=job_counts.index,
        y=job_counts.values
    )

    # Layout for the bar graph
    layout = go.Layout(title=f'Job Titles Distribution for {company}',
                       xaxis={'title': 'Job Titles'},
                       yaxis={'title': 'Total Job Postings'})

    return {'data': [bar_data], 'layout': layout}

## Part 7

In [102]:
# Extract unique countries from the dataset
countries = df['Country Name'].dropna().unique()
selected_aspects = [
    'Access to electricity (% of population)',
    'Employment in services (% of total employment) (modeled ILO estimate)',
    'Agriculture value added (% of GDP)',
    'Unemployment with advanced education (% of total labor force with advanced education)',
    'Life expectancy at birth total (years)'
]

def update_comparison_chart(selected_country, selected_aspect):
    # Filter dataset based on selected country and aspect
    filtered_data = df[(df['Country Name'] == selected_country) & (df['Series Name'] == selected_aspect)]

    # Extract years and corresponding values for the selected aspect (from '1995 [YR1995]' to '2016 [YR2016]')
    years = [f"{year} [YR{year}]" for year in range(1995, 2017)]
    values = filtered_data[years].values.tolist()[0]  # Assuming only one row matches

    # Create line plot for the selected aspect
    fig7 = px.line(
        x=years,
        y=values,
        labels={
            'x': 'Year',
            'y': selected_aspect
        },
        title=f'{selected_aspect} in {selected_country} (1995 to 2016)',
        markers=True,
    )
    return fig7

## Part 8

In [103]:
def update_output(country, section):
    filtered_df = df_growth[df_growth['country_name']==country]
    filtered_df = filtered_df[filtered_df['isic_section_name']==section]
    melted = filtered_df.melt(id_vars=['industry_name'], value_vars=['2015','2016','2017','2018','2019'], var_name='year', value_name='growth_rate')
    fig = px.line(
            melted,
            x="year",
            y="growth_rate",
            color="industry_name",
            markers=True,
            title="Growth rate over the years",
        )
    fig.add_hline(y=0, line=dict(color='red',dash='dash'))
    i = 0
    for trace in fig.data:
        i += 1
        if i > 3:
            trace.update(visible='legendonly')
    fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'])
    return fig

## Part 9

In [104]:
df_cm = df_mig['Country Migration']
df_im = df_mig['Industry Migration']
df_sm = df_mig['Skill Migration']
df_country = df_cm[['base_country_code','base_lat','base_long']].drop_duplicates(subset=['base_country_code'])
df_im = pd.merge(df_im, df_country, left_on='country_code', right_on='base_country_code').drop('base_country_code',axis=1)
df_sm = pd.merge(df_sm, df_country, left_on='country_code', right_on='base_country_code').drop('base_country_code',axis=1)
df_im['avg_migration'] = df_im[['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018','net_per_10K_2019']].mean(axis=1)
df_sm['avg_migration'] = df_sm[['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018','net_per_10K_2019']].mean(axis=1)
df_im['size'] = np.abs(df_im['avg_migration'])
df_sm['size'] = np.abs(df_sm['avg_migration'])

In [105]:
def geo_update(isoption, sel):
    options = []
    filtered_df = None
    if isoption == 'industry migration':
        options=[{"label": s, "value": s}
                for s in df_im['industry_name'].dropna().unique()]
        filtered_df = df_im[df_im['industry_name']==sel]
    elif isoption == 'skill migration':
        options=[{"label": s, "value": s}
                for s in df_sm['skill_group_name'].dropna().unique()]
        filtered_df = df_sm[df_sm['skill_group_name']==sel]
    fig = px.scatter_geo(
        data_frame= filtered_df,
        lat="base_lat",
        lon="base_long",
        size="size",
        hover_name="country_name",
        projection='equirectangular',color='avg_migration',
                      color_continuous_scale=[(0, 'red'), (0.5, 'yellow'), (1, 'green')],
                      color_continuous_midpoint=0
    )
    fig.update_layout(
    margin=dict(l=20, r=20, t=10, b=20),  # Adjust margins
    font=dict(color=colors['text']),
    paper_bgcolor=colors['background'],
    coloraxis_colorbar=dict(title='Average inflow per 10K')
    )
    return options,fig

## Dash Code

In [106]:
# styles
colors = {
    'background': '#d9ffb3',
    'text': '#0066cc',
    'title' : '#000000'
}
def_stylesheet = [
        {
            'selector': 'node',
            'style': {
                'background-color': 'grey',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'node[type = \'industry\']',
            'style': {
                'background-color': '#ff66cc',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'edge',
            'style': {
                'line-color': 'grey',  # Default color for edges
                'opacity':'0.15'
            }
        }]

In [107]:
app = Dash(
    meta_tags=[
        {'name': 'viewport', 'content': 'width=device-width, initial-scale=1.0'}
    ],
    # external_stylesheets=['..\Code\global.css']
)

In [108]:
# Define callback to update Part 1
@app.callback(
    Output('bar-chart', 'figure'),
    Input('search-button', 'n_clicks'),
    Input('search-input', 'value'),
    Input('country-input', 'value'),
)
def update_bar_chart(n_clicks, search_value, country_value):
    if n_clicks:
        update_fig1_plot(search_value, country_value)
    return fig1

In [109]:
# Define callback to update Part 3
@app.callback(
    Output("fig1", "figure"),
    Input("sector-filter", "value"),
    Input("critique-filter", "value")
)
def update_fig3_Sect_Ratings(sector, crit):
    return update_Sect_Ratings(sector, crit)

In [110]:
# Define callback to update Part 4
@app.callback(
    [Output("cytoscape", "elements"),
     Output('cytoscape', 'stylesheet')],
    [Input("sector-filter-1", "value"),
     Input("skill-filter-1", "value"),
     Input('cytoscape', 'tapNodeData')]
)
def update_fig4_style(section, skill, selected_node):
    return update_style(section, skill, selected_node)

In [111]:
# Define callback to update Part 5
@app.callback(
    Output('state-dropdown', 'options'),
    [Input('country-options-dropdown', 'value')]
)
def update_state_dropdown(selected_country):
    if selected_country is None:
        return []
    state_options = [{'label': state, 'value': state} for state in linkedin_job_postings[linkedin_job_postings['search_country'] == selected_country]['search_city'].unique()]
    return state_options

# Define callback to update histogram based on selected country and state
@app.callback(
    Output('histogram', 'figure'),
    [Input('country-options-dropdown', 'value'),
     Input('state-dropdown', 'value')]
)
def update_fig5_histogram(selected_country, selected_state):
    return update_histogram(selected_country, selected_state)

In [112]:
# Define callback to update Part 6
@app.callback(
    Output('company-comparison-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_fig6_chart(company):
    return update_chart(company)

In [113]:
# Define callback to update Part 7
@app.callback(
    Output('aspect-comparison-chart', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('aspect-dropdown', 'value')]
)
def update_comparison_chart_fig7(selected_country, selected_aspect):
    return update_comparison_chart(selected_country, selected_aspect)

In [114]:
# Define callback to update Part 8
@app.callback(
    Output("growth-line", "figure"),
    [Input("country-filter-2", "value"),
    Input("sector-filter-2", "value")]
)
def update_fig8_output(country, section):
    return update_output(country, section)

In [115]:
# Define callback to update Part 9
@app.callback(
    [Output("filter-3","options"),
    Output("geo", "figure")],
    [Input("i-s", "value"),
    Input("filter-3", "value")]
)
def geo_fig9_update(isoption, sel):
    return geo_update(isoption, sel)

In [116]:
button_style = {
    'background-color': '#FFFFFF',
    'border': '1px solid #222222',
    'border-radius': '8px',
    'box-sizing': 'border-box',
    'color': '#222222',
    'cursor': 'pointer',
    'display': 'inline-block',
    'font-family': 'Circular, -apple-system, BlinkMacSystemFont, Roboto, "Helvetica Neue", sans-serif',
    'font-size': '16px',
    'font-weight': '600',
    'line-height': '20px',
    'margin': '10px',
    'outline': 'none',
    'padding': '13px 23px',
    'position': 'relative',
    'text-align': 'center',
    'text-decoration': 'none',
    'touch-action': 'manipulation',
    'transition': 'box-shadow .2s, -ms-transform .1s, -webkit-transform .1s, transform .1s',
    'user-select': 'none',
    '-webkit-user-select': 'none',
    'width': 'auto',
    'height':'40px'
}

In [195]:
# Define app layout
app.layout = html.Div([
    html.Div(children=[
        html.H1('Dashboard Menu'),
    ],style={
        'display':'flex',
        'background-color':'rgba(30,37,46,255)',
        'align-items':'center',
        'height':'100px',
        'justify-content':'center',
        'margin-bottom':'10px'
    },className='dashboard1'),
    html.Div([
        dcc.Tabs([
            dcc.Tab(label='Tab one', children=[
                # fig2 Layout
                html.Div(
                    [
                        html.Div(
                            [
                                html.H1(
                                    children='Top 10 cities with highest jobs in each country',
                                    style={
                                        'textAlign': 'center',
                                        'color': colors['title']
                                    }
                                ),
                                dcc.Graph(figure=fig2)
                            ],
                            style={
                                'margin-top': '10px',
                                'margin-bottom': '10px'
                            })

                    ],
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'height': '1230px',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                    }),
            ]),
            dcc.Tab(label='Tab two', children=[
                # fig1 Layoout
                html.Div(
                    [
                        html.H1(
                            children='Top companies based off your loaction and job title',
                            style={
                                'textAlign': 'center',
                                'color': colors['title']
                            }
                        ),
                        html.Div(children=[
                            dcc.Dropdown(
                                id='country-input',
                                options=[{'label': i, 'value': i}
                                         for i in linkedin_job_postings['search_country'].unique()],
                                value='United States',
                                style={'width': '300px', 'height': '30px'}
                            ),
                            html.Div([
                                dcc.Input(
                                    id='search-input',
                                    type='text',
                                    placeholder='Search for job titles...',
                                    debounce=True,
                                    style={'width': '300px', 'height': '25px', 'margin': '10px',
                                           'padding': '5px', 'border': '1px solid #ccc'}
                                ),
                                html.Button('Search', id='search-button',
                                            style=button_style, className='button-6')
                            ])
                        ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),
                        html.Div([
                            dcc.Graph(figure=fig1, id='bar-chart')
                        ], style={'margin-top': '20px'}),

                    ],
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'height': '1200px',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                    }
                ),
            ]),
            dcc.Tab(label='Tab four', children=[
                # fig3 Layout
                html.Div(
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                    }, children=[
                        html.H1(
                            children='Companies based off your likings',
                            style={
                                'textAlign': 'center',
                                'color': colors['title']
                            }
                        ),
                        dbc.Row([
                            html.Div(children=[
                                html.H3("Sector",style={
                                    'margin':'10px',
                                    'margin-right':'0px'
                                },className='labelfour'),
                                dcc.Dropdown(
                                    id="sector-filter",
                                    options=[{"label": s, "value": s}
                                            for s in df_twc['sector'].dropna().unique()],
                                    value=None,
                                    multi=True,
                                    style={
                                        "width": "250px",
                                        "height": "50%",
                                        "color": "#000",
                                        "fontFamily": "sans-serif",
                                        'margin':'10px',
                                        'margin-right':'80px'
                                    }
                                ),
                                html.H3("Features",style={
                                    'margin':'10px',
                                    'margin-right':'0px'
                                },className='labelfour'),
                                dcc.Dropdown(
                                    id="critique-filter",
                                    options=[{"label": l, "value": l}
                                            for l in union_HRF],
                                    value=None,
                                    multi=True,
                                    style={
                                        "width": "250px",
                                        "height": "50%",
                                        "color": "#000",
                                        "fontFamily": "sans-serif",
                                        'margin':'10px'
                                    }
                                ),
                            ],style={
                                'display':'flex',
                                'align-items':'center',
                                'justify-content':'center'
                            }
                            ),
                            dcc.Graph(id="fig1")
                        ])
                    ]),
            ]),
            dcc.Tab(label='Tab nine', children=[
                # fig6 Layout
                html.Div([
                    html.H1(
                        children='Job title distribution of your companies',
                        style={
                            'textAlign': 'center',
                            'color': colors['title']
                        }
                    ),
                    html.Div(children=[
                        html.H3('Choose Company',style={
                            'margin':'10px'
                        }),
                        dcc.Dropdown(
                            id='company-dropdown',
                            options=[{'label': company, 'value': company}
                                    for company in selected_company],
                            value=selected_company[0],
                            style={
                                "width": "250px",
                                "height": "50%",
                                "color": "#000",
                                "fontFamily": "sans-serif",
                                'margin':'10px',
                            }
                        ),
                    ], style={
                        'display':'flex',
                        'align-items':'center',
                        'justify-content':'center'
                    }),
                    dcc.Graph(id='company-comparison-chart')
                ],
                    style={
                    'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                    'border-radius': '5px',
                    'padding': '20px',
                    'background-color': 'rgba(255, 255, 255, 1.0)',
                    'margin-top': '10px',
                    'margin-bottom': '10px'
                }),
            ]),
            dcc.Tab(label='Tab seven', children=[
                # fig5 Layout
                html.Div([
                    html.H1(
                        children='Top 20 job positions searched in the city',
                        style={
                            'textAlign': 'center',
                            'color': colors['title']
                        }
                    ),
                    html.Div(children=[
                        html.Div("Select a country:", style={
                                 'margin-top': '35px', 'font-weight': 'bold', 'font-size': '20px'}),
                        dcc.Dropdown(
                            id='country-options-dropdown',
                            options=country_options,
                            value=default_country,
                            style={'width': '300px', 'height': '30px',
                                   'margin': '10px', 'padding': '5px', },
                            placeholder='Select a country'
                        ),
                        html.Div("Select a city:", style={
                                 'margin-left': '40px', 'margin-top': '35px', 'font-weight': 'bold', 'font-size': '20px'}),
                        dcc.Dropdown(
                            id='state-dropdown',
                            value=default_city,
                            style={'width': '300px', 'height': '30px',
                                   'margin': '10px', 'padding': '5px', },
                            placeholder='Select a state'
                        )
                    ], style={'margin-bottom': '20px', 'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),

                    dcc.Graph(id='histogram', style={'margin-top': '10px', })
                ],
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                }
                ),
            ]),
            dcc.Tab(label='Tab six', children=[
                # fig4 Layout
                html.Div(
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                    }, children=[
                        html.H1(
                            children='Industry Skill Needs',
                            style={
                                'textAlign': 'center',
                                'color': colors['title'],
                                'margin-bottom':'20px'
                            }
                        ),

                        dbc.Row([
                            html.Div([
                                html.H3("Sector",style={'margin':'10px'}),
                                dcc.Dropdown(
                                    id="sector-filter-1",
                                    options=[{"label": s, "value": s}
                                             for s in df_ldd['isic_section_name'].dropna().unique()],
                                    value='Information and communication',
                                    style={
                                        "width": "75%",  # set the width of the dropdown to 50% of its container
                                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                                        # set the background color of the dropdown to dark gray
                                        "color": "#000",  # set the font color of the dropdown to light gray
                                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                                    }
                                ),
                                html.H3("Skills",style={'margin':'10px'}),
                                dcc.Dropdown(
                                    id="skill-filter-1",
                                    options=[{"label": s, "value": s}
                                             for s in df_ldd['skill_group_name'].dropna().unique()],
                                    value=None,
                                    multi=True,
                                    style={
                                        "width": "75%",  # set the width of the dropdown to 50% of its container
                                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                                        # set the background color of the dropdown to dark gray
                                        "color": "#000",  # set the font color of the dropdown to light gray
                                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                                    }
                                )],style={
                                    'margin':'10px',
                                    'display':'flex'
                                }),
                            cyto.Cytoscape(
                                id='cytoscape',
                                layout={'name': 'circle'},
                                style={'height': '700px', 'width': '100%',
                                       'label': 'data(label)'},
                                stylesheet=def_stylesheet
                            )
                        ])
                    ]),
            ]),

            dcc.Tab(label='Tab seven', children=[
                # fig7 Layout
                html.Div([
                    html.H1(children="Country Evaluation Dashboard", style={
                            'textAlign': 'center',
                            'color': colors['title']
                            }),
                         html.Div(children=[
                             html.Div("Select a country:", style={
                                      'margin-top': '35px', 'font-weight': 'bold', 'font-size': '20px'}),
                             dcc.Dropdown(id='country-dropdown', options=[{'label': country, 'value': country}
                                          for country in countries], value=countries[0], style={'width': '300px', 'height': '30px',
                                   'margin': '10px', 'padding': '5px', },),
                             html.Div("Select criteria:", style={
                                      'margin-left': '10px', 'margin-top': '35px', 'font-weight': 'bold', 'font-size': '20px'}),
                             dcc.Dropdown(id='aspect-dropdown', options=[{'label': aspect, 'value': aspect}
                                          for aspect in selected_aspects], value=selected_aspects[0], style={'width': '300px', 'height': '30px',
                                   'margin': '10px', 'padding': '5px', },)


                         ], style={'margin-bottom': '20px', 'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),

                    dcc.Graph(id='aspect-comparison-chart')
                         ],
                    style={
                    'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                    'border-radius': '5px',
                    'padding': '20px',
                    'margin-bottom': '50px',
                    'margin-top': '10px',
                })
            ]),

            dcc.Tab(label='Tab eight', children=[
                # fig8 Layout
                html.Div(
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                    }
                    , children=[
                    html.H1(
                        children='Industry Growth Rates',
                        style={
                            'textAlign': 'center',
                            'color': colors['title']
                        }
                    ),
                    dbc.Row([
                        html.Div(children=[
                            html.H3("Country",style={'margin':'10px'}),
                            dcc.Dropdown(
                                id="country-filter-2",
                                options=[{"label": s, "value": s}
                                        for s in df_growth['country_name'].dropna().unique()],
                                value='India',
                                style={
                                    "width": "75%",  # set the width of the dropdown to 50% of its container
                                    "height": "50%",  # set the height of each dropdown item to 40 pixels
                                    # set the background color of the dropdown to dark gray
                                    "color": "#000",  # set the font color of the dropdown to light gray
                                    "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                                }
                            ),
                            html.H3("Sector", style={'margin':'10px'}),
                            dcc.Dropdown(
                                id="sector-filter-2",
                                options=[{"label": s, "value": s}
                                        for s in df_growth['isic_section_name'].dropna().unique()],
                                value='Information and communication',
                                style={
                                    "width": "75%",  # set the width of the dropdown to 50% of its container
                                    "height": "50%",  # set the height of each dropdown item to 40 pixels
                                    # set the background color of the dropdown to dark gray
                                    "color": "#000",  # set the font color of the dropdown to light gray
                                    "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                                }
                            )
                        ],style={
                            'margin':'10px',
                            'display':'flex'
                        })
                        ,
                        dcc.Graph(id="growth-line")

                    ])
                ])
            ]),

            dcc.Tab(label='Tab last', children=[
                # fig9 Layout
                html.Div(
                    style={
                        'box-shadow': '10px 10px 10px 10px rgba(0,0,0,0.2)',
                        'border-radius': '5px',
                        'padding': '20px',
                        'background-color': 'rgba(255, 255, 255, 1.0)',
                        'margin-top': '10px',
                        'margin-bottom': '10px'
                    }
                    ,children=[
                        html.H1(
                            children='Talent Migration',
                            style={
                                'textAlign': 'center',
                                'color': colors['title']
                            }
                        ),

                        dbc.Row([
                            html.Div(children=[
                                dcc.RadioItems(
                                    id="i-s",
                                    inline=True,
                                    options=["   Industry Migration ", "   Skill Migration   "],
                                    value="industry migration",
                                    style={
                                        'font-size': '20px',  # set the font family of the dropdown to sans-serif
                                        'margin':'10px'
                                    }
                                ),
                                html.H3("Select from options",style={'margin':'10px'}),
                                dcc.Dropdown(
                                    id="filter-3",
                                    #                     options=[{"label": s, "value": s}
                                    #                              for s in df_growth['isic_section_name'].dropna().unique()],
                                    value=None,
                                    style={
                                        "width": "75%",  # set the width of the dropdown to 50% of its container
                                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                                        # set the background color of the dropdown to dark gray
                                        "color": "#000",  # set the font color of the dropdown to light gray
                                        "fontFamily": "sans-serif",  # set the font family of the dropdown to sans-serif
                                        "padding": "20",
                                    }
                                ),
                            ]),
                            
                            dcc.Graph(id="geo")
                        ])
                ])
            ]),
        ]),
    ], style={
        'align-items': 'center',
        'justify-content': 'center',
        'width': '90%',
        'margin-left': '5%',
        'margin-right': '5%',
    })
],
    style={
    'margin': '0',
    'padding': '0',
    'box-sizing': 'border-box',
    'width': '100%',
    'height': '100%',
    'background-color':'rgba(250,250,250,255)'
})


In [196]:
Timer(1, open_browser).start()
app.run_server(debug=True, port=8080)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.